In [2]:
import pandas as pd


In [3]:
dataset_url = 'https://raw.githubusercontent.com/mateuschaves/water-data-mining/main/water_potability.csv'
data = pd.read_csv(dataset_url)
data.head()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890455,20791.318981,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.057858,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.541732,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.417441,8.059332,356.886136,363.266516,18.436524,100.341674,4.628771,0
4,9.092223,181.101509,17978.986339,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0


In [4]:
data.describe()

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
count,2785.000000,3276.000000,3276.000000,3276.000000,2495.000000,3276.000000,3276.000000,3114.000000,3276.000000,3276.000000
mean,7.080795,196.369496,22014.092526,7.122277,333.775777,426.205111,14.284970,66.396293,3.966786,0.390110
std,1.594320,32.879761,8768.570828,1.583085,41.416840,80.824064,3.308162,16.175008,0.780382,0.487849
min,0.000000,47.432000,320.942611,0.352000,129.000000,181.483754,2.200000,0.738000,1.450000,0.000000
25%,6.093092,176.850538,15666.690297,6.127421,307.699498,365.734414,12.065801,55.844536,3.439711,0.000000
50%,7.036752,196.967627,20927.833607,7.130299,333.073546,421.884968,14.218338,66.622485,3.955028,0.000000
75%,8.062066,216.667456,27332.762127,8.114887,359.950170,481.792304,16.557652,77.337473,4.500320,1.000000
max,14.000000,323.124000,61227.196008,13.127000,481.030642,753.342620,28.300000,124.000000,6.739000,1.000000


**As colunas ```ph```, ```Trihalomethanes``` e ```Sulfate``` possuem valores vazios**

In [8]:
pip install https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip (25.9 MB)


In [10]:
import pandas_profiling

SyntaxError: ignored

In [11]:
profile = pandas_profiling.ProfileReport(data)
profile

NameError: ignored

**Eliminando valores `NaN`**

In [12]:
# Remover todas as linhas com valores faltando
# 1265 linhas foram eliminadas
data_drop_na = data.dropna()

# Preencher os valores ausentes com 0
data_fill_zero_na = data.fillna(0)


# Preenchendo os valores ausentes com a média
ph_average = data['ph'].sum() / data['ph'].count()
trihalomethanes_average = data['Trihalomethanes'].sum() / data['Trihalomethanes'].count()
sulfate_average = data['Sulfate'].sum() / data['Sulfate'].count()

ph_column = data['ph'].fillna(ph_average)
trihalomethanes_average = data['Trihalomethanes'].fillna(trihalomethanes_average)
sulfate_column = data['Sulfate'].fillna(sulfate_average)

data_with_average = data

data_with_average['ph'] = ph_column
data_with_average['Trihalomethanes'] = trihalomethanes_average
data_with_average['Sulfate'] = sulfate_column

**Dataset desbalanceado**

In [13]:
data['Potability'].value_counts()

0    1998
1    1278
Name: Potability, dtype: int64

**Balanceando o dataset**

In [14]:
from imblearn.under_sampling import RandomUnderSampler

undersample = RandomUnderSampler(sampling_strategy='majority')

features = data.iloc[:,:-1]
classes = data.iloc[:,-1]

under_feature_data, under_classes_data = undersample.fit_resample(features, classes)


under_feature_data['Potability'] = under_classes_data

under_data = under_feature_data

under_data['Potability'].value_counts()



1    1278
0    1278
Name: Potability, dtype: int64

**Encontrando valores duplicados**

In [15]:
under_data[under_data.duplicated()]

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability


In [17]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
import matplotlib.pyplot as plt
import seaborn as sns

ImportError: ignored

**Treinando modelo com conjunto de dados sem nenhum tratamento**

In [18]:
features_cols = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']

X = data[features_cols]
Y = data.Potability

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)

clf = clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print(metrics.confusion_matrix(y_test, y_pred))

print(metrics.classification_report(y_test, y_pred))

[[546  39]
 [325  73]]
              precision    recall  f1-score   support

           0       0.63      0.93      0.75       585
           1       0.65      0.18      0.29       398

    accuracy                           0.63       983
   macro avg       0.64      0.56      0.52       983
weighted avg       0.64      0.63      0.56       983



**Treinando modelo com conjunto de dados dropando os valores `NaN` e tamanho do teste de `10%`**

In [19]:
features_cols = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']

X = data_drop_na[features_cols]
Y = data_drop_na.Potability

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)

clf = clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print(metrics.confusion_matrix(y_test, y_pred))

print(metrics.classification_report(y_test, y_pred))

[[352  24]
 [198  30]]
              precision    recall  f1-score   support

           0       0.64      0.94      0.76       376
           1       0.56      0.13      0.21       228

    accuracy                           0.63       604
   macro avg       0.60      0.53      0.49       604
weighted avg       0.61      0.63      0.55       604



**Treinando modelo com conjunto de dados preenchendos os valores `NaN` com `0`** 

In [20]:
features_cols = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']

X = data_fill_zero_na[features_cols]
Y = data_fill_zero_na.Potability

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)

clf = clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print(metrics.confusion_matrix(y_test, y_pred))

print(metrics.classification_report(y_test, y_pred))

[[574  11]
 [380  18]]
              precision    recall  f1-score   support

           0       0.60      0.98      0.75       585
           1       0.62      0.05      0.08       398

    accuracy                           0.60       983
   macro avg       0.61      0.51      0.42       983
weighted avg       0.61      0.60      0.48       983



**Treinando modelo com conjunto de dados preenchendos os valores `NaN` com a média** 

In [21]:
features_cols = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']

X = data_with_average[features_cols]
Y = data_with_average.Potability

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)

clf = clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print(metrics.confusion_matrix(y_test, y_pred))

print(metrics.classification_report(y_test, y_pred))

[[546  39]
 [325  73]]
              precision    recall  f1-score   support

           0       0.63      0.93      0.75       585
           1       0.65      0.18      0.29       398

    accuracy                           0.63       983
   macro avg       0.64      0.56      0.52       983
weighted avg       0.64      0.63      0.56       983



**Treinando modelo com conjunto de dados balanceado utilizando Undersampler** 

In [22]:
features_cols = ['ph', 'Hardness', 'Solids', 'Chloramines', 'Sulfate', 'Conductivity', 'Organic_carbon', 'Trihalomethanes', 'Turbidity']

X = under_data[features_cols]
Y = under_data.Potability

X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=1)

clf = DecisionTreeClassifier(criterion="entropy", max_depth=3)

clf = clf.fit(X_train,y_train)

y_pred = clf.predict(X_test)

print(metrics.confusion_matrix(y_test, y_pred))

print(metrics.classification_report(y_test, y_pred))

[[328  47]
 [304  88]]
              precision    recall  f1-score   support

           0       0.52      0.87      0.65       375
           1       0.65      0.22      0.33       392

    accuracy                           0.54       767
   macro avg       0.59      0.55      0.49       767
weighted avg       0.59      0.54      0.49       767



# Utilizando o classificador Adaboost 

In [25]:
from sklearn.ensemble import AdaBoostClassifier

adabooster_classifier = AdaBoostClassifier(
    base_estimator=None, n_estimators=50, learning_rate=1.0,
    algorithm='SAMME.R', random_state=None
)

adabooster_classifier.fit(X_train, y_train)


y_pred = adabooster_classifier.predict(X_test)

print(metrics.confusion_matrix(y_test, y_pred))

print(metrics.classification_report(y_test, y_pred))

[[205 170]
 [199 193]]
              precision    recall  f1-score   support

           0       0.51      0.55      0.53       375
           1       0.53      0.49      0.51       392

    accuracy                           0.52       767
   macro avg       0.52      0.52      0.52       767
weighted avg       0.52      0.52      0.52       767



# Utilizando o classificador Random Forest

In [29]:
from sklearn.ensemble import RandomForestClassifier

random_forest_classifier = RandomForestClassifier(
  bootstrap=True, class_weight=None, criterion='gini',
  max_depth=None, max_features='auto', max_leaf_nodes=None,
  min_impurity_decrease=0.0,
  min_samples_leaf=1, min_samples_split=2,
  min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
  oob_score=False, random_state=None, verbose=0,
  warm_start=False
)

random_forest_classifier.fit(X_train, y_train)

y_pred = random_forest_classifier.predict(X_test)

print(metrics.confusion_matrix(y_test, y_pred))

print(metrics.classification_report(y_test, y_pred))

[[248 127]
 [181 211]]
              precision    recall  f1-score   support

           0       0.58      0.66      0.62       375
           1       0.62      0.54      0.58       392

    accuracy                           0.60       767
   macro avg       0.60      0.60      0.60       767
weighted avg       0.60      0.60      0.60       767

